In [1]:
import torch
import torch.nn as nn
from standard_LLM import TinyTransformer
from dataset import get_batch_split, vocab_size, stoi, decode

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
device = (
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: mps


In [5]:
block_size = 64
batch_size = 64
n_embd = 128
n_heads = 4
n_layers = 4

steps = 1000
eval_interval = 500
eval_steps = 50

model = TinyTransformer(vocab_size, n_embd=n_embd, block_size=block_size, n_heads=n_heads, n_layers=n_layers).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)
loss_fn = nn.CrossEntropyLoss()

@torch.no_grad()
def estimate_loss():
    model.eval()
    out = {}
    for split in ['train', 'val']:
        losses = torch.zeros(eval_steps)
        for k in range(eval_steps):
            x, y = get_batch_split(split, block_size=block_size, batch_size=batch_size, device=device)
            logits = model(x)
            B, T, C = logits.shape
            loss = loss_fn(logits.view(B*T, C), y.view(B*T))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

for step in range(steps):
    x, y = get_batch_split('train', block_size=block_size, batch_size=batch_size, device=device)

    logits = model(x)
    B, T, C = logits.shape
    loss = loss_fn(logits.view(B*T, C), y.view(B*T))

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 
    optimizer.step()
    
    if step % eval_interval == 0 or step == steps - 1:
        losses = estimate_loss()
        print(f"Step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

torch.save(model.state_dict(), 'tiny_shakespeare_model.pth')
print("Model saved to tiny_shakespeare_model.pth")


Step 0: train loss 3.8883, val loss 3.9024
Step 500: train loss 2.0917, val loss 2.1292
Step 999: train loss 1.8006, val loss 1.9069
Model saved to tiny_shakespeare_model.pth


In [7]:
def generate(model, start, length=100, temperature=0.8, top_k=40):
    model.eval()
    with torch.no_grad():
        x = torch.tensor([stoi[s] for s in start], dtype=torch.long).unsqueeze(0).to(device)
        idx = model.generate(x, max_new_tokens=length, temperature=temperature, top_k=top_k)
        return decode(idx[0].tolist())

print(generate(model, "ROMEO:", length=200))

ROMEO:
Chomme, pot this I'll dow!

MESTERO:
Chase have a gantly for vingman forself.

VOBRINGSIA:
The fort the off there rad ears, and the him prize,
That this it fromoutth at buch I may.

Sher:
Sho I he, y
